In [82]:
import vl_convert as vlc
import json
import copy
import pandas as pd

In [83]:
normalize={'date':'date','Date':'date','day':'date','Day':'date','month':'month','Month':'month','year':'year','Year':'year'}
time_format={'date':'D','Date':'D','day':'D','Day':'D','month':'M','Month':'M','year':'Y','Year':'Y'}
def get_opposite_color(hex_color):
	hex_color = hex_color.lstrip('#')
	r = int(hex_color[0:2], 16)
	g = int(hex_color[2:4], 16)
	b = int(hex_color[4:6], 16)
	r = 255 - r
	g = 255 - g
	b = 255 - b
	r_hex = format(r, '02x')
	g_hex = format(g, '02x')
	b_hex = format(b, '02x')
	return f"#{r_hex}{g_hex}{b_hex}"
def remove_lonely_numbers(lst):
	result = []
	lst=sorted(lst)
	for i in range(len(lst)):
		if (i == 0):
			if (lst[i+1] - lst[i] == 1):  
				result.append(lst[i])
		if (i == len(lst) - 1):
			if(lst[i] - lst[i-1] == 1):
				result.append(lst[i])
		if  (i!=0 and i!=len(lst)-1):
			if (lst[i] - lst[i-1] == 1) | (lst[i+1] - lst[i] == 1):
				result.append(lst[i])
	return result

def generate_conditions(numlist,mainField):
	filtered_years = remove_lonely_numbers(numlist)
	print(filtered_years)
	ranges = []
	start = None
	for year in filtered_years:
		if start is None:
			start = year
		elif year + 1 not in filtered_years:
			end = year
			if start != end:
				ranges.append((start, end))
			start = None
	if start is not None:
		end = filtered_years[-1]
		if start != end:
			ranges.append((start, end))
	conditions = [f"({normalize[mainField]}(datum['{mainField}']) >= {start-1} && {normalize[mainField]}(datum['{mainField}']) <= {end-1})" for start, end in ranges ]
	return conditions

def ScatterMark(vega,mainField, subField, mainType, subType, value1,value2,mark=0,angle=0):
	# newcolor = get_opposite_color(vega["encoding"]["color"]["value"])
	newcolor='red'
	if mark==0:
		if mainType == 'temporal':
			# vega["data"]["format"]= {"type": "csv", "parse": {f'{mainField}': f"date:'%{time_format[mainField]}'"}}
			vega["layer"].append( {
			"mark": {"type": "point", "filled": True},
			"transform": [{"filter": f"{normalize[mainField]}(datum['{mainField}']) == {value1} && {normalize[subField]}(datum['{subField}']) == {value2}"}],
			"encoding": {
				"color": {"value": newcolor},
				"x": {"type": mainType, "field": mainField},
				"y": {"type": subType, "field": subField}
			}})
		else:
			vega["layer"].append( {
				"mark": {"type": "point", "filled": True},
				"transform": [{"filter": f"datum['{mainField}'] == {value1} && datum['{subField}'] == {value2}"}], 
				"encoding": {
					"color": {"value": newcolor},
					"size": {"value": 100},
					"x": {"type": mainType, "field": mainField, "axis": {}, "bin": False},
					"y": {"type": subType, "field": subField}
				}})
	else:
		vega["layer"].append( {
				"mark": {"type": "text", 'angle':angle},
				"transform": [{"filter": f"datum['{mainField}'] == {value1} && datum['{subField}'] == {value2}"}], 
				"encoding": {
			"text" :{"value": mark},
			"color": {"value": newcolor},
			"x": {"type": mainType, "field": mainField},
			"y": {"type": subType, "field": subField},
			"fontSize":{"value":100}
		}})
	return vega

def ScatterText(vega,mainField, subField, mainType, subType, value1,value2, text,degree=0,dy=-10):
	# newcolor = get_opposite_color(vega["encoding"]["color"]["value"])
	newcolor='black'
	size=14
	if degree!=0:
		size=100
	if mainType == 'temporal':
		vega["data"]["format"]= {"type": "csv", "parse": {f'{mainField}': f"date:'%{time_format[mainField]}'"}}
		vega["layer"].append( {
		"mark": {"type": "text", "filled": True,"fontSize":size,'angle':degree,'dy':dy},
		"transform": [{"filter": f"{normalize[mainField]}(datum['{mainField}']) == {value1} && {normalize[subField]}(datum['{subField}']) == {value2}"}],
		"encoding": {
			"text" :{"value": text},
			"color": {"value": newcolor},
			"x": {"type": mainType, "field": mainField},
			"y": {"type": subType, "field": subField},
			"fontSize":{"value":size}
		}})
	else:
		vega["layer"].append( {
			"mark": {"type": "text", "filled": True,"fontsize":size,'angle':degree,'dy':dy},
			"transform": [{"filter": f"datum['{mainField}'] == {value1} && datum['{subField}'] == {value2}"}], # type: ignore
			"encoding": {
				"text" :{"value": text},
				"color": {"value": newcolor},
				"x": {"type": mainType, "field": mainField},
				"y": {"type": subType, "field": subField},
				"fontSize":{"value":size}
			}})
	return vega
def ScatterLine(vega,mainField, subField, mainType, subType, value,value1=0,subValue=0,subValue1=0):
	# newcolor = get_opposite_color(vega["encoding"]["color"]["value"])
	newcolor='red'
	if subValue==0:
		if vega["encoding"]["x"]["field"] == subField:
			vega["layer"].append({ "data": { "values": [{}] }, "mark": { "type": "rule", "color": newcolor }, "encoding": { "x": { "datum": value } } })
			if value1!=0:
				vega["layer"].append({ "data": { "values": [{}] }, "mark": { "type": "rule", "color": newcolor }, "encoding": { "x": { "datum": value1 } } })
		else:
			vega["layer"].append({ "data": { "values": [{}] }, "mark": { "type": "rule", "color": newcolor }, "encoding": { "y": { "datum": value } } })
			if value1!=0:
				vega["layer"].append({ "data": { "values": [{}] }, "mark": { "type": "rule", "color": newcolor }, "encoding": { "y": { "datum": value1 } } })
	else:
		vega["layer"].append({
	  "data": {
		"values": [
		  {"PlotX1": value, "PlotY1": value1, "PlotX2": subValue, "PlotY2": subValue1}
		]
	  },
	  "mark": {
		"type": "rule",
		"stroke": newcolor
	  },
	  "encoding": {
		"x": {"field": "PlotX1", "type": mainType},
		"y": {"field": "PlotY1", "type": subType},
		"x2": {"field": "PlotX2", "type": mainType},
		"y2": {"field": "PlotY2", "type": subType}
	  }
	})

	return vega
def ScatterColor(vega,mainField, subField, mainType, subType, value, condition,value1=0):
	# newcolor = get_opposite_color(vega["encoding"]["color"]["value"])
	newcolor='red'
	new_layer = copy.deepcopy(vega["layer"][0])
	if condition == 'equal':
		new_layer["transform"] = [{"filter": f"datum['{mainField}'] == {value} && datum['{subField}'] == {value1}"}]
	elif condition == 'above':
		new_layer["transform"] = [{"filter": f"datum['{subField}'] >= {value}"}]
	elif condition == 'below':
		new_layer["transform"] = [{"filter": f"datum['{subField}'] <= {value}"}]
	elif condition == 'above_and_below':
		new_layer["transform"] = [{"filter": f"datum['{subField}'] >= {value} && datum['{subField}'] <= {value1}"}]
	elif condition == 'left_bound':
		new_layer["transform"] = [{"filter": f"datum['{mainField}'] >= {value}"}]
	elif condition == 'right_bound':
		new_layer["transform"] = [{"filter": f"datum['{mainField}'] <= {value}"}]
	elif condition == 'left_and_right':
		new_layer["transform"] = [{"filter": f"datum['{mainField}'] >= {value} && datum['{mainField}'] <= {value1}"}]
	elif condition == 'compare':
		new_layer["transform"] = [{"filter": f"datum['{mainField}'] == {value} || datum['{mainField}'] == {value1}"}]

	new_layer["encoding"]["color"]={"value": 'red'}
	vega["layer"].append(new_layer)
	return vega
def ScatterOpacity(vega,mainField, subField, mainType, subType, value,condition,value1=0):
	vega["layer"][0]["encoding"]["opacity"]={"value": 0.1}
	new_layer = copy.deepcopy(vega["layer"][0])
	if condition == 'equal':
		new_layer["transform"] = [{"filter": f"datum['{mainField}'] == {value} && datum['{subField}'] == {value1}"}]
	elif condition == 'above':
		new_layer["transform"] = [{"filter": f"datum['{subField}'] >= {value}"}]
	elif condition == 'below':
		new_layer["transform"] = [{"filter": f"datum['{subField}'] <= {value}"}]
	elif condition == 'above_and_below':
		new_layer["transform"] = [{"filter": f"datum['{subField}'] >= {value} && datum['{subField}'] <= {value1}"}]
	elif condition == 'left_bound':
		new_layer["transform"] = [{"filter": f"datum['{mainField}'] >= {value}"}]
	elif condition == 'right_bound':
		new_layer["transform"] = [{"filter": f"datum['{mainField}'] <= {value}"}]
	elif condition == 'left_and_right':
		new_layer["transform"] = [{"filter": f"datum['{mainField}'] >= {value} && datum['{mainField}'] <= {value1}"}]
	elif condition == 'compare':
		new_layer["transform"] = [{"filter": f"datum['{mainField}'] == {value} || datum['{mainField}'] == {value1}"}]
				
	new_layer["encoding"]["opacity"]={"value": 1}
	vega["layer"].append(new_layer)
	return vega

def ScatterArea(vega,mainField, subField, mainType, subType, value,value1=0,subValue=0,subValue1=0,shade=True):
	# newcolor = get_opposite_color(vega["encoding"]["color"]["value"])
	newcolor='red'
	# df = pd.read_csv(vega["data"]["url"])
	if shade==True:
		stroke=0
		opacity=0.2
	else:
		stroke=2
		opacity=0

	vega["layer"].append({
	  "data": {
		"values": [
		  {"PlotX1": value, "PlotY1": value1, "PlotX2": subValue, "PlotY2": subValue1}
		]
	  },
		"mark": {"type": "rect", "color": newcolor,"stroke":newcolor,"fillOpacity":opacity,"strokeWidth":stroke},
	  "encoding": {
		"x": {"field": "PlotX1", "type": mainType},
		"y": {"field": "PlotY1", "type": subType},
		"x2": {"field": "PlotX2", "type": mainType},
		"y2": {"field": "PlotY2", "type": subType}
	}})
	return vega

def ScatterRegression(vega,mainField,subField,width=2,alpha=1,box=False):
	newcolor='red'
	new_layer = copy.deepcopy(vega["layer"][0])
	new_layer['encoding']['color']= {"value":newcolor}
	new_layer['transform']=[{"regression": subField,"on": mainField}]
	new_layer['mark']='line'
	new_layer['encoding']["strokeWidth"]={"value":width}
	new_layer['encoding']["opacity"]={"value": alpha}
	if box:
		new_new_layer=copy.deepcopy(new_layer)
		new_new_layer['encoding']["strokeWidth"]={"value":width-2}
		new_new_layer['encoding']['color']= {"value":'white'}
		vega["layer"]=[new_layer,new_new_layer,vega["layer"][0]]
	else:
		vega["layer"].append(new_layer)
	return vega

def ScatterTrendColor(vega,mainField,subField,mainType,subType):
	newcolor='red'
	new_layer =vega["layer"][0]
	new_layer['encoding']['color']= {"field":mainField,"type":mainType,"scale": {"scheme": "blueorange"}}
	return vega

def ScatterTrendOpacity(vega,mainField,subField,mainType,subType):
	newcolor='red'
	new_layer =vega["layer"][0]
	new_layer['encoding']['color']= {"field":mainField,"type":mainType}
	return vega

In [84]:
vl_spec=r'''
{
	"$schema": "https://vega.github.io/schema/vega-lite/v5.json",
	"data": {"url": "https://raw.githubusercontent.com/vega/vega-datasets/main/data/cars.json"},
	"mark": "circle",
	"encoding": {
		"x": {"field": "Horsepower", "type": "quantitative"},
		"y": {"field": "Miles_per_Gallon", "type": "quantitative"}
	},
    "title":"The horsepower and MPG of different cars"
}
'''

In [85]:
png_data = vlc.vegalite_to_png(vl_spec=vl_spec, scale=2)
with open("Sample-Scatter-RetrieveValue.png", "wb") as f:
	f.write(png_data)

In [86]:
# vl_json=json.loads(vl_spec)
# vl_json["layer"] = [{ 'mark': vl_json['mark'], 'encoding': vl_json['encoding'] }]
# vl_json=ScatterLine(vl_json, "Horsepower", "Miles_per_Gallon", "quantitative", "quantitative", 10)
# del vl_json['mark'], vl_json['encoding']
# print(vl_json)
# png_data = vlc.vegalite_to_png(vl_spec=vl_json, scale=2)
# with open("RetrieveValue-Scatter-Line.png", "wb") as f:
# 	f.write(png_data)

In [87]:
vl_json=json.loads(vl_spec)
vl_json["layer"] = [{ 'mark': vl_json['mark'], 'encoding': vl_json['encoding'] }]
vl_json=ScatterMark(vl_json, "Horsepower", "Miles_per_Gallon", "quantitative", "quantitative", 165, 15)
vl_json=ScatterText(vl_json, "Horsepower", "Miles_per_Gallon", "quantitative", "quantitative", 165, 15,'A')
# vl_json=ScatterMark(vl_json, "Horsepower", "Miles_per_Gallon", "quantitative", "quantitative", 150, 18)
# vl_json=ScatterText(vl_json, "Horsepower", "Miles_per_Gallon", "quantitative", "quantitative", 150, 18,'B')
del vl_json['mark'], vl_json['encoding']
print(vl_json)
png_data = vlc.vegalite_to_png(vl_spec=vl_json, scale=2)
with open("RetrieveValue-Scatter-Mark.png", "wb") as f:
	f.write(png_data)

{'$schema': 'https://vega.github.io/schema/vega-lite/v5.json', 'data': {'url': 'https://raw.githubusercontent.com/vega/vega-datasets/main/data/cars.json'}, 'title': 'The horsepower and MPG of different cars', 'layer': [{'mark': 'circle', 'encoding': {'x': {'field': 'Horsepower', 'type': 'quantitative'}, 'y': {'field': 'Miles_per_Gallon', 'type': 'quantitative'}}}, {'mark': {'type': 'point', 'filled': True}, 'transform': [{'filter': "datum['Horsepower'] == 165 && datum['Miles_per_Gallon'] == 15"}], 'encoding': {'color': {'value': 'red'}, 'size': {'value': 100}, 'x': {'type': 'quantitative', 'field': 'Horsepower', 'axis': {}, 'bin': False}, 'y': {'type': 'quantitative', 'field': 'Miles_per_Gallon'}}}, {'mark': {'type': 'text', 'filled': True, 'fontsize': 14, 'angle': 0, 'dy': -10}, 'transform': [{'filter': "datum['Horsepower'] == 165 && datum['Miles_per_Gallon'] == 15"}], 'encoding': {'text': {'value': 'A'}, 'color': {'value': 'black'}, 'x': {'type': 'quantitative', 'field': 'Horsepower'

In [88]:
vl_json=json.loads(vl_spec)
vl_json["layer"] = [{ 'mark': vl_json['mark'], 'encoding': vl_json['encoding'] }]
vl_json=ScatterOpacity(vl_json, "Horsepower", "Miles_per_Gallon", "quantitative", "quantitative", 165, 'equal',15)
vl_json=ScatterText(vl_json, "Horsepower", "Miles_per_Gallon", "quantitative", "quantitative", 165, 15,'A')
del vl_json['mark'], vl_json['encoding']
print(vl_json)
png_data = vlc.vegalite_to_png(vl_spec=vl_json, scale=2)
with open("RetrieveValue-Scatter-Opacity.png", "wb") as f:
	f.write(png_data)

{'$schema': 'https://vega.github.io/schema/vega-lite/v5.json', 'data': {'url': 'https://raw.githubusercontent.com/vega/vega-datasets/main/data/cars.json'}, 'title': 'The horsepower and MPG of different cars', 'layer': [{'mark': 'circle', 'encoding': {'x': {'field': 'Horsepower', 'type': 'quantitative'}, 'y': {'field': 'Miles_per_Gallon', 'type': 'quantitative'}, 'opacity': {'value': 0.1}}}, {'mark': 'circle', 'encoding': {'x': {'field': 'Horsepower', 'type': 'quantitative'}, 'y': {'field': 'Miles_per_Gallon', 'type': 'quantitative'}, 'opacity': {'value': 1}}, 'transform': [{'filter': "datum['Horsepower'] == 165 && datum['Miles_per_Gallon'] == 15"}]}, {'mark': {'type': 'text', 'filled': True, 'fontsize': 14, 'angle': 0, 'dy': -10}, 'transform': [{'filter': "datum['Horsepower'] == 165 && datum['Miles_per_Gallon'] == 15"}], 'encoding': {'text': {'value': 'A'}, 'color': {'value': 'black'}, 'x': {'type': 'quantitative', 'field': 'Horsepower'}, 'y': {'type': 'quantitative', 'field': 'Miles_p

In [89]:
vl_json=json.loads(vl_spec)
vl_json["layer"] = [{ 'mark': vl_json['mark'], 'encoding': vl_json['encoding'] }]
vl_json=ScatterColor(vl_json, "Horsepower", "Miles_per_Gallon", "quantitative", "quantitative", 165,'equal',15)
vl_json=ScatterText(vl_json, "Horsepower", "Miles_per_Gallon", "quantitative", "quantitative", 165, 15,'A')
del vl_json['mark'], vl_json['encoding']
print(vl_json)
png_data = vlc.vegalite_to_png(vl_spec=vl_json, scale=2)
with open("RetrieveValue-Scatter-Color.png", "wb") as f:
	f.write(png_data)

{'$schema': 'https://vega.github.io/schema/vega-lite/v5.json', 'data': {'url': 'https://raw.githubusercontent.com/vega/vega-datasets/main/data/cars.json'}, 'title': 'The horsepower and MPG of different cars', 'layer': [{'mark': 'circle', 'encoding': {'x': {'field': 'Horsepower', 'type': 'quantitative'}, 'y': {'field': 'Miles_per_Gallon', 'type': 'quantitative'}}}, {'mark': 'circle', 'encoding': {'x': {'field': 'Horsepower', 'type': 'quantitative'}, 'y': {'field': 'Miles_per_Gallon', 'type': 'quantitative'}, 'color': {'value': 'red'}}, 'transform': [{'filter': "datum['Horsepower'] == 165 && datum['Miles_per_Gallon'] == 15"}]}, {'mark': {'type': 'text', 'filled': True, 'fontsize': 14, 'angle': 0, 'dy': -10}, 'transform': [{'filter': "datum['Horsepower'] == 165 && datum['Miles_per_Gallon'] == 15"}], 'encoding': {'text': {'value': 'A'}, 'color': {'value': 'black'}, 'x': {'type': 'quantitative', 'field': 'Horsepower'}, 'y': {'type': 'quantitative', 'field': 'Miles_per_Gallon'}, 'fontSize': 

In [90]:
vl_json=json.loads(vl_spec)
vl_json["layer"] = [{ 'mark': vl_json['mark'], 'encoding': vl_json['encoding'] }]
vl_json=ScatterText(vl_json, "Horsepower", "Miles_per_Gallon", "quantitative", "quantitative", 165, 15,'165,15')
vl_json=ScatterText(vl_json, "Horsepower", "Miles_per_Gallon", "quantitative", "quantitative", 165, 15,'A',dy=10)
del vl_json['mark'], vl_json['encoding']
print(vl_json)
png_data = vlc.vegalite_to_png(vl_spec=vl_json, scale=2)
with open("RetrieveValue-Scatter-Text.png", "wb") as f:
	f.write(png_data)

{'$schema': 'https://vega.github.io/schema/vega-lite/v5.json', 'data': {'url': 'https://raw.githubusercontent.com/vega/vega-datasets/main/data/cars.json'}, 'title': 'The horsepower and MPG of different cars', 'layer': [{'mark': 'circle', 'encoding': {'x': {'field': 'Horsepower', 'type': 'quantitative'}, 'y': {'field': 'Miles_per_Gallon', 'type': 'quantitative'}}}, {'mark': {'type': 'text', 'filled': True, 'fontsize': 14, 'angle': 0, 'dy': -10}, 'transform': [{'filter': "datum['Horsepower'] == 165 && datum['Miles_per_Gallon'] == 15"}], 'encoding': {'text': {'value': '165,15'}, 'color': {'value': 'black'}, 'x': {'type': 'quantitative', 'field': 'Horsepower'}, 'y': {'type': 'quantitative', 'field': 'Miles_per_Gallon'}, 'fontSize': {'value': 14}}}, {'mark': {'type': 'text', 'filled': True, 'fontsize': 14, 'angle': 0, 'dy': 10}, 'transform': [{'filter': "datum['Horsepower'] == 165 && datum['Miles_per_Gallon'] == 15"}], 'encoding': {'text': {'value': 'A'}, 'color': {'value': 'black'}, 'x': {

In [91]:
vl_json=json.loads(vl_spec)
vl_json["layer"] = [{ 'mark': vl_json['mark'], 'encoding': vl_json['encoding'] }]
vl_json=ScatterLine(vl_json, "Horsepower", "Miles_per_Gallon", "quantitative", "quantitative", 15)
vl_json=ScatterLine(vl_json, "Horsepower", "Miles_per_Gallon", "quantitative", "quantitative", 165,0,165,50)
del vl_json['mark'], vl_json['encoding']
print(vl_json)
png_data = vlc.vegalite_to_png(vl_spec=vl_json, scale=2)
with open("RetrieveValue-Scatter-Line.png", "wb") as f:
	f.write(png_data)

{'$schema': 'https://vega.github.io/schema/vega-lite/v5.json', 'data': {'url': 'https://raw.githubusercontent.com/vega/vega-datasets/main/data/cars.json'}, 'title': 'The horsepower and MPG of different cars', 'layer': [{'mark': 'circle', 'encoding': {'x': {'field': 'Horsepower', 'type': 'quantitative'}, 'y': {'field': 'Miles_per_Gallon', 'type': 'quantitative'}}}, {'data': {'values': [{}]}, 'mark': {'type': 'rule', 'color': 'red'}, 'encoding': {'y': {'datum': 15}}}, {'data': {'values': [{'PlotX1': 165, 'PlotY1': 0, 'PlotX2': 165, 'PlotY2': 50}]}, 'mark': {'type': 'rule', 'stroke': 'red'}, 'encoding': {'x': {'field': 'PlotX1', 'type': 'quantitative'}, 'y': {'field': 'PlotY1', 'type': 'quantitative'}, 'x2': {'field': 'PlotX2', 'type': 'quantitative'}, 'y2': {'field': 'PlotY2', 'type': 'quantitative'}}}]}


In [92]:
vl_spec=r''' 
{
  "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
  "description": "A scatterplot showing body mass and flipper lengths of penguins.",
  "data": {
    "url": "data/penguins.json"
  },
  "mark": "point",
  "encoding": {
    "x": {
      "field": "Flipper Length (mm)",
      "type": "quantitative",
      "scale": {"zero": false}
    },
    "y": {
      "field": "Body Mass (g)",
      "type": "quantitative",
      "scale": {"zero": false}
    }
  },
  "title":"A scatterplot showing body mass and flipper lengths of penguins"
}
'''

In [93]:
png_data = vlc.vegalite_to_png(vl_spec=vl_spec, scale=2)
with open("Sample-Scatter-Filter.png", "wb") as f:
	f.write(png_data)

In [94]:
vl_json=json.loads(vl_spec)
vl_json["layer"] = [{ 'mark': vl_json['mark'], 'encoding': vl_json['encoding'] }]
vl_json=ScatterColor(vl_json, "Flipper Length (mm)", "Body Mass (g)", "quantitative", "quantitative", 5000,'above')
# vl_json=ScatterText(vl_json, "Horsepower", "Miles_per_Gallon", "quantitative", "quantitative", 165, 15,'A')
del vl_json['mark'], vl_json['encoding']
print(vl_json)
png_data = vlc.vegalite_to_png(vl_spec=vl_json, scale=2)
with open("Filter-Scatter-Color.png", "wb") as f:
	f.write(png_data)

{'$schema': 'https://vega.github.io/schema/vega-lite/v5.json', 'description': 'A scatterplot showing body mass and flipper lengths of penguins.', 'data': {'url': 'data/penguins.json'}, 'title': 'A scatterplot showing body mass and flipper lengths of penguins', 'layer': [{'mark': 'point', 'encoding': {'x': {'field': 'Flipper Length (mm)', 'type': 'quantitative', 'scale': {'zero': False}}, 'y': {'field': 'Body Mass (g)', 'type': 'quantitative', 'scale': {'zero': False}}}}, {'mark': 'point', 'encoding': {'x': {'field': 'Flipper Length (mm)', 'type': 'quantitative', 'scale': {'zero': False}}, 'y': {'field': 'Body Mass (g)', 'type': 'quantitative', 'scale': {'zero': False}}, 'color': {'value': 'red'}}, 'transform': [{'filter': "datum['Body Mass (g)'] >= 5000"}]}]}


In [95]:
vl_json=json.loads(vl_spec)
vl_json["layer"] = [{ 'mark': vl_json['mark'], 'encoding': vl_json['encoding'] }]
vl_json=ScatterOpacity(vl_json, "Flipper Length (mm)", "Body Mass (g)", "quantitative", "quantitative", 5000,'above')
# vl_json=ScatterText(vl_json, "Horsepower", "Miles_per_Gallon", "quantitative", "quantitative", 165, 15,'A')
del vl_json['mark'], vl_json['encoding']
print(vl_json)
png_data = vlc.vegalite_to_png(vl_spec=vl_json, scale=2)
with open("Filter-Scatter-Opacity.png", "wb") as f:
	f.write(png_data)

{'$schema': 'https://vega.github.io/schema/vega-lite/v5.json', 'description': 'A scatterplot showing body mass and flipper lengths of penguins.', 'data': {'url': 'data/penguins.json'}, 'title': 'A scatterplot showing body mass and flipper lengths of penguins', 'layer': [{'mark': 'point', 'encoding': {'x': {'field': 'Flipper Length (mm)', 'type': 'quantitative', 'scale': {'zero': False}}, 'y': {'field': 'Body Mass (g)', 'type': 'quantitative', 'scale': {'zero': False}}, 'opacity': {'value': 0.1}}}, {'mark': 'point', 'encoding': {'x': {'field': 'Flipper Length (mm)', 'type': 'quantitative', 'scale': {'zero': False}}, 'y': {'field': 'Body Mass (g)', 'type': 'quantitative', 'scale': {'zero': False}}, 'opacity': {'value': 1}}, 'transform': [{'filter': "datum['Body Mass (g)'] >= 5000"}]}]}


In [96]:
vl_json=json.loads(vl_spec)
vl_json["layer"] = [{ 'mark': vl_json['mark'], 'encoding': vl_json['encoding'] }]
vl_json=ScatterLine(vl_json, "Flipper Length (mm)", "Body Mass (g)", "quantitative", "quantitative", 5000)
del vl_json['mark'], vl_json['encoding']
print(vl_json)
png_data = vlc.vegalite_to_png(vl_spec=vl_json, scale=2)
with open("Filter-Scatter-Line.png", "wb") as f:
	f.write(png_data)

{'$schema': 'https://vega.github.io/schema/vega-lite/v5.json', 'description': 'A scatterplot showing body mass and flipper lengths of penguins.', 'data': {'url': 'data/penguins.json'}, 'title': 'A scatterplot showing body mass and flipper lengths of penguins', 'layer': [{'mark': 'point', 'encoding': {'x': {'field': 'Flipper Length (mm)', 'type': 'quantitative', 'scale': {'zero': False}}, 'y': {'field': 'Body Mass (g)', 'type': 'quantitative', 'scale': {'zero': False}}}}, {'data': {'values': [{}]}, 'mark': {'type': 'rule', 'color': 'red'}, 'encoding': {'y': {'datum': 5000}}}]}


In [97]:
vl_json=json.loads(vl_spec)
vl_json["layer"] = [{ 'mark': vl_json['mark'], 'encoding': vl_json['encoding'] }]
vl_json=ScatterArea(vl_json, "Horsepower", "Miles_per_Gallon", "quantitative", "quantitative", 170,5000,240,6500)
del vl_json['mark'], vl_json['encoding']
print(vl_json)
png_data = vlc.vegalite_to_png(vl_spec=vl_json, scale=2)
with open("Filter-Scatter-Shade.png", "wb") as f:
	f.write(png_data)

{'$schema': 'https://vega.github.io/schema/vega-lite/v5.json', 'description': 'A scatterplot showing body mass and flipper lengths of penguins.', 'data': {'url': 'data/penguins.json'}, 'title': 'A scatterplot showing body mass and flipper lengths of penguins', 'layer': [{'mark': 'point', 'encoding': {'x': {'field': 'Flipper Length (mm)', 'type': 'quantitative', 'scale': {'zero': False}}, 'y': {'field': 'Body Mass (g)', 'type': 'quantitative', 'scale': {'zero': False}}}}, {'data': {'values': [{'PlotX1': 170, 'PlotY1': 5000, 'PlotX2': 240, 'PlotY2': 6500}]}, 'mark': {'type': 'rect', 'color': 'red', 'stroke': 'red', 'fillOpacity': 0.2, 'strokeWidth': 0}, 'encoding': {'x': {'field': 'PlotX1', 'type': 'quantitative'}, 'y': {'field': 'PlotY1', 'type': 'quantitative'}, 'x2': {'field': 'PlotX2', 'type': 'quantitative'}, 'y2': {'field': 'PlotY2', 'type': 'quantitative'}}}]}


In [98]:
vl_json=json.loads(vl_spec)
vl_json["layer"] = [{ 'mark': vl_json['mark'], 'encoding': vl_json['encoding'] }]
vl_json=ScatterArea(vl_json, "Horsepower", "Miles_per_Gallon", "quantitative", "quantitative",170,5000,240,6500,shade=False)
del vl_json['mark'], vl_json['encoding']
print(vl_json)
png_data = vlc.vegalite_to_png(vl_spec=vl_json, scale=2)
with open("Filter-Scatter-Bound.png", "wb") as f:
	f.write(png_data)

{'$schema': 'https://vega.github.io/schema/vega-lite/v5.json', 'description': 'A scatterplot showing body mass and flipper lengths of penguins.', 'data': {'url': 'data/penguins.json'}, 'title': 'A scatterplot showing body mass and flipper lengths of penguins', 'layer': [{'mark': 'point', 'encoding': {'x': {'field': 'Flipper Length (mm)', 'type': 'quantitative', 'scale': {'zero': False}}, 'y': {'field': 'Body Mass (g)', 'type': 'quantitative', 'scale': {'zero': False}}}}, {'data': {'values': [{'PlotX1': 170, 'PlotY1': 5000, 'PlotX2': 240, 'PlotY2': 6500}]}, 'mark': {'type': 'rect', 'color': 'red', 'stroke': 'red', 'fillOpacity': 0, 'strokeWidth': 2}, 'encoding': {'x': {'field': 'PlotX1', 'type': 'quantitative'}, 'y': {'field': 'PlotY1', 'type': 'quantitative'}, 'x2': {'field': 'PlotX2', 'type': 'quantitative'}, 'y2': {'field': 'PlotY2', 'type': 'quantitative'}}}]}


In [99]:
vl_spec=r''' 
{
    "$schema": "https://vega.github.io/schema/vega-lite/v3.4.0.json",
    "data": {
      "url": "https://vega.github.io/vega-datasets/data/gapminder-health-income.csv"
    },
    "config": {"view": {"width": 400, "height": 300}},
    "mark": "point",
    "encoding": {
      "x": {"type": "quantitative", "field": "income"},
      "y": {"type": "quantitative", "field": "health"}
    },
    "title":"The income and life expectancy of different countries"
  }
'''

In [100]:
png_data = vlc.vegalite_to_png(vl_spec=vl_spec, scale=2)
with open("Sample-Scatter-Range.png", "wb") as f:
	f.write(png_data)

In [101]:
vl_json=json.loads(vl_spec)
vl_json["layer"] = [{ 'mark': vl_json['mark'], 'encoding': vl_json['encoding'] }]
vl_json=ScatterColor(vl_json, "income", "health", "quantitative", "quantitative", 0,'left_and_right',50000)
# vl_json=ScatterText(vl_json, "Horsepower", "Miles_per_Gallon", "quantitative", "quantitative", 165, 15,'A')
del vl_json['mark'], vl_json['encoding']
print(vl_json)
png_data = vlc.vegalite_to_png(vl_spec=vl_json, scale=2)
with open("Range-Scatter-Color.png", "wb") as f:
	f.write(png_data)

{'$schema': 'https://vega.github.io/schema/vega-lite/v3.4.0.json', 'data': {'url': 'https://vega.github.io/vega-datasets/data/gapminder-health-income.csv'}, 'config': {'view': {'width': 400, 'height': 300}}, 'title': 'The income and life expectancy of different countries', 'layer': [{'mark': 'point', 'encoding': {'x': {'type': 'quantitative', 'field': 'income'}, 'y': {'type': 'quantitative', 'field': 'health'}}}, {'mark': 'point', 'encoding': {'x': {'type': 'quantitative', 'field': 'income'}, 'y': {'type': 'quantitative', 'field': 'health'}, 'color': {'value': 'red'}}, 'transform': [{'filter': "datum['income'] >= 0 && datum['income'] <= 50000"}]}]}


In [102]:
vl_json=json.loads(vl_spec)
vl_json["layer"] = [{ 'mark': vl_json['mark'], 'encoding': vl_json['encoding'] }]
vl_json=ScatterOpacity(vl_json, "income", "health", "quantitative", "quantitative", 0,'left_and_right',50000)
# vl_json=ScatterText(vl_json, "Horsepower", "Miles_per_Gallon", "quantitative", "quantitative", 165, 15,'A')
del vl_json['mark'], vl_json['encoding']
print(vl_json)
png_data = vlc.vegalite_to_png(vl_spec=vl_json, scale=2)
with open("Range-Scatter-Opacity.png", "wb") as f:
	f.write(png_data)

{'$schema': 'https://vega.github.io/schema/vega-lite/v3.4.0.json', 'data': {'url': 'https://vega.github.io/vega-datasets/data/gapminder-health-income.csv'}, 'config': {'view': {'width': 400, 'height': 300}}, 'title': 'The income and life expectancy of different countries', 'layer': [{'mark': 'point', 'encoding': {'x': {'type': 'quantitative', 'field': 'income'}, 'y': {'type': 'quantitative', 'field': 'health'}, 'opacity': {'value': 0.1}}}, {'mark': 'point', 'encoding': {'x': {'type': 'quantitative', 'field': 'income'}, 'y': {'type': 'quantitative', 'field': 'health'}, 'opacity': {'value': 1}}, 'transform': [{'filter': "datum['income'] >= 0 && datum['income'] <= 50000"}]}]}


In [103]:
vl_json=json.loads(vl_spec)
vl_json["layer"] = [{ 'mark': vl_json['mark'], 'encoding': vl_json['encoding'] }]
# vl_json=ScatterLine(vl_json, "income", "health", "quantitative", "quantitative", 0,0,0,90)
vl_json=ScatterLine(vl_json, "income", "health", "quantitative", "quantitative", 50000,0,50000,90)
del vl_json['mark'], vl_json['encoding']
print(vl_json)
png_data = vlc.vegalite_to_png(vl_spec=vl_json, scale=2)
with open("Range-Scatter-Line.png", "wb") as f:
	f.write(png_data)

{'$schema': 'https://vega.github.io/schema/vega-lite/v3.4.0.json', 'data': {'url': 'https://vega.github.io/vega-datasets/data/gapminder-health-income.csv'}, 'config': {'view': {'width': 400, 'height': 300}}, 'title': 'The income and life expectancy of different countries', 'layer': [{'mark': 'point', 'encoding': {'x': {'type': 'quantitative', 'field': 'income'}, 'y': {'type': 'quantitative', 'field': 'health'}}}, {'data': {'values': [{'PlotX1': 50000, 'PlotY1': 0, 'PlotX2': 50000, 'PlotY2': 90}]}, 'mark': {'type': 'rule', 'stroke': 'red'}, 'encoding': {'x': {'field': 'PlotX1', 'type': 'quantitative'}, 'y': {'field': 'PlotY1', 'type': 'quantitative'}, 'x2': {'field': 'PlotX2', 'type': 'quantitative'}, 'y2': {'field': 'PlotY2', 'type': 'quantitative'}}}]}


In [104]:
vl_json=json.loads(vl_spec)
vl_json["layer"] = [{ 'mark': vl_json['mark'], 'encoding': vl_json['encoding'] }]
vl_json=ScatterArea(vl_json, "income", "health", "quantitative", "quantitative", 0,50,50000,90)
del vl_json['mark'], vl_json['encoding']
print(vl_json)
png_data = vlc.vegalite_to_png(vl_spec=vl_json, scale=2)
with open("Range-Scatter-Shade.png", "wb") as f:
	f.write(png_data)

{'$schema': 'https://vega.github.io/schema/vega-lite/v3.4.0.json', 'data': {'url': 'https://vega.github.io/vega-datasets/data/gapminder-health-income.csv'}, 'config': {'view': {'width': 400, 'height': 300}}, 'title': 'The income and life expectancy of different countries', 'layer': [{'mark': 'point', 'encoding': {'x': {'type': 'quantitative', 'field': 'income'}, 'y': {'type': 'quantitative', 'field': 'health'}}}, {'data': {'values': [{'PlotX1': 0, 'PlotY1': 50, 'PlotX2': 50000, 'PlotY2': 90}]}, 'mark': {'type': 'rect', 'color': 'red', 'stroke': 'red', 'fillOpacity': 0.2, 'strokeWidth': 0}, 'encoding': {'x': {'field': 'PlotX1', 'type': 'quantitative'}, 'y': {'field': 'PlotY1', 'type': 'quantitative'}, 'x2': {'field': 'PlotX2', 'type': 'quantitative'}, 'y2': {'field': 'PlotY2', 'type': 'quantitative'}}}]}


In [105]:
vl_json=json.loads(vl_spec)
vl_json["layer"] = [{ 'mark': vl_json['mark'], 'encoding': vl_json['encoding'] }]
vl_json=ScatterArea(vl_json, "income", "health", "quantitative", "quantitative", 0,50,50000,90,shade=False)
del vl_json['mark'], vl_json['encoding']
print(vl_json)
png_data = vlc.vegalite_to_png(vl_spec=vl_json, scale=2)
with open("Range-Scatter-Bound.png", "wb") as f:
	f.write(png_data)

{'$schema': 'https://vega.github.io/schema/vega-lite/v3.4.0.json', 'data': {'url': 'https://vega.github.io/vega-datasets/data/gapminder-health-income.csv'}, 'config': {'view': {'width': 400, 'height': 300}}, 'title': 'The income and life expectancy of different countries', 'layer': [{'mark': 'point', 'encoding': {'x': {'type': 'quantitative', 'field': 'income'}, 'y': {'type': 'quantitative', 'field': 'health'}}}, {'data': {'values': [{'PlotX1': 0, 'PlotY1': 50, 'PlotX2': 50000, 'PlotY2': 90}]}, 'mark': {'type': 'rect', 'color': 'red', 'stroke': 'red', 'fillOpacity': 0, 'strokeWidth': 2}, 'encoding': {'x': {'field': 'PlotX1', 'type': 'quantitative'}, 'y': {'field': 'PlotY1', 'type': 'quantitative'}, 'x2': {'field': 'PlotX2', 'type': 'quantitative'}, 'y2': {'field': 'PlotY2', 'type': 'quantitative'}}}]}


In [106]:
vl_spec=r''' 
{
  "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
  "data": {"url": "data/movies.json"},
  "mark": {"type": "point", "filled": true},
  "encoding": {
    "x": {"field": "Rotten Tomatoes Rating", "type": "quantitative"},
    "y": {"field": "IMDB Rating", "type": "quantitative"}
  },
  "title": "Relation between Rotten Tomatoes and IMDB rating"
}
'''

In [107]:
png_data = vlc.vegalite_to_png(vl_spec=vl_spec, scale=2)
with open("Sample-Scatter-Trend.png", "wb") as f:
	f.write(png_data)

In [108]:
vl_json=json.loads(vl_spec)
vl_json["layer"] = [{ 'mark': vl_json['mark'], 'encoding': vl_json['encoding'] }]
vl_json=ScatterRegression(vl_json, "Rotten Tomatoes Rating", "IMDB Rating")
del vl_json['mark'], vl_json['encoding']
print(vl_json)
png_data = vlc.vegalite_to_png(vl_spec=vl_json, scale=2)
with open("Trend-Scatter-Line.png", "wb") as f:
	f.write(png_data)

{'$schema': 'https://vega.github.io/schema/vega-lite/v5.json', 'data': {'url': 'data/movies.json'}, 'title': 'Relation between Rotten Tomatoes and IMDB rating', 'layer': [{'mark': {'type': 'point', 'filled': True}, 'encoding': {'x': {'field': 'Rotten Tomatoes Rating', 'type': 'quantitative'}, 'y': {'field': 'IMDB Rating', 'type': 'quantitative'}}}, {'mark': 'line', 'encoding': {'x': {'field': 'Rotten Tomatoes Rating', 'type': 'quantitative'}, 'y': {'field': 'IMDB Rating', 'type': 'quantitative'}, 'color': {'value': 'red'}, 'strokeWidth': {'value': 2}, 'opacity': {'value': 1}}, 'transform': [{'regression': 'IMDB Rating', 'on': 'Rotten Tomatoes Rating'}]}]}


In [109]:
vl_json=json.loads(vl_spec)
vl_json["layer"] = [{ 'mark': vl_json['mark'], 'encoding': vl_json['encoding'] }]
vl_json=ScatterRegression(vl_json, "Rotten Tomatoes Rating", "IMDB Rating",40,0.2)
del vl_json['mark'], vl_json['encoding']
print(vl_json)
png_data = vlc.vegalite_to_png(vl_spec=vl_json, scale=2)
with open("Trend-Scatter-Shade.png", "wb") as f:
	f.write(png_data)

{'$schema': 'https://vega.github.io/schema/vega-lite/v5.json', 'data': {'url': 'data/movies.json'}, 'title': 'Relation between Rotten Tomatoes and IMDB rating', 'layer': [{'mark': {'type': 'point', 'filled': True}, 'encoding': {'x': {'field': 'Rotten Tomatoes Rating', 'type': 'quantitative'}, 'y': {'field': 'IMDB Rating', 'type': 'quantitative'}}}, {'mark': 'line', 'encoding': {'x': {'field': 'Rotten Tomatoes Rating', 'type': 'quantitative'}, 'y': {'field': 'IMDB Rating', 'type': 'quantitative'}, 'color': {'value': 'red'}, 'strokeWidth': {'value': 40}, 'opacity': {'value': 0.2}}, 'transform': [{'regression': 'IMDB Rating', 'on': 'Rotten Tomatoes Rating'}]}]}


In [110]:
vl_json=json.loads(vl_spec)
vl_json["layer"] = [{ 'mark': vl_json['mark'], 'encoding': vl_json['encoding'] }]
vl_json=ScatterRegression(vl_json, "Rotten Tomatoes Rating", "IMDB Rating",40,1,box=True)
del vl_json['mark'], vl_json['encoding']
print(vl_json)
png_data = vlc.vegalite_to_png(vl_spec=vl_json, scale=2)
with open("Trend-Scatter-Bound.png", "wb") as f:
	f.write(png_data)

{'$schema': 'https://vega.github.io/schema/vega-lite/v5.json', 'data': {'url': 'data/movies.json'}, 'title': 'Relation between Rotten Tomatoes and IMDB rating', 'layer': [{'mark': 'line', 'encoding': {'x': {'field': 'Rotten Tomatoes Rating', 'type': 'quantitative'}, 'y': {'field': 'IMDB Rating', 'type': 'quantitative'}, 'color': {'value': 'red'}, 'strokeWidth': {'value': 40}, 'opacity': {'value': 1}}, 'transform': [{'regression': 'IMDB Rating', 'on': 'Rotten Tomatoes Rating'}]}, {'mark': 'line', 'encoding': {'x': {'field': 'Rotten Tomatoes Rating', 'type': 'quantitative'}, 'y': {'field': 'IMDB Rating', 'type': 'quantitative'}, 'color': {'value': 'white'}, 'strokeWidth': {'value': 38}, 'opacity': {'value': 1}}, 'transform': [{'regression': 'IMDB Rating', 'on': 'Rotten Tomatoes Rating'}]}, {'mark': {'type': 'point', 'filled': True}, 'encoding': {'x': {'field': 'Rotten Tomatoes Rating', 'type': 'quantitative'}, 'y': {'field': 'IMDB Rating', 'type': 'quantitative'}}}]}


In [111]:
vl_json=json.loads(vl_spec)
vl_json["layer"] = [{ 'mark': vl_json['mark'], 'encoding': vl_json['encoding'] }]
vl_json=ScatterTrendColor(vl_json, "Rotten Tomatoes Rating", "IMDB Rating","quantitative","quantitative")
del vl_json['mark'], vl_json['encoding']
print(vl_json)
png_data = vlc.vegalite_to_png(vl_spec=vl_json, scale=2)
with open("Trend-Scatter-Color.png", "wb") as f:
	f.write(png_data)

{'$schema': 'https://vega.github.io/schema/vega-lite/v5.json', 'data': {'url': 'data/movies.json'}, 'title': 'Relation between Rotten Tomatoes and IMDB rating', 'layer': [{'mark': {'type': 'point', 'filled': True}, 'encoding': {'x': {'field': 'Rotten Tomatoes Rating', 'type': 'quantitative'}, 'y': {'field': 'IMDB Rating', 'type': 'quantitative'}, 'color': {'field': 'Rotten Tomatoes Rating', 'type': 'quantitative', 'scale': {'scheme': 'blueorange'}}}}]}


In [112]:
vl_json=json.loads(vl_spec)
vl_json["layer"] = [{ 'mark': vl_json['mark'], 'encoding': vl_json['encoding'] }]
vl_json=ScatterTrendOpacity(vl_json, "Rotten Tomatoes Rating", "IMDB Rating","quantitative","quantitative")
del vl_json['mark'], vl_json['encoding']
print(vl_json)
png_data = vlc.vegalite_to_png(vl_spec=vl_json, scale=2)
with open("Trend-Scatter-Opacity.png", "wb") as f:
	f.write(png_data)

{'$schema': 'https://vega.github.io/schema/vega-lite/v5.json', 'data': {'url': 'data/movies.json'}, 'title': 'Relation between Rotten Tomatoes and IMDB rating', 'layer': [{'mark': {'type': 'point', 'filled': True}, 'encoding': {'x': {'field': 'Rotten Tomatoes Rating', 'type': 'quantitative'}, 'y': {'field': 'IMDB Rating', 'type': 'quantitative'}, 'color': {'field': 'Rotten Tomatoes Rating', 'type': 'quantitative'}}}]}


In [113]:
vl_json=json.loads(vl_spec)
vl_json["layer"] = [{ 'mark': vl_json['mark'], 'encoding': vl_json['encoding'] }]
vl_json=ScatterMark(vl_json, "Rotten Tomatoes Rating", "IMDB Rating","quantitative","quantitative",50,6,"→",-45)
del vl_json['mark'], vl_json['encoding']
print(vl_json)
png_data = vlc.vegalite_to_png(vl_spec=vl_json, scale=2)
with open("Trend-Scatter-Mark.png", "wb") as f:
	f.write(png_data)

{'$schema': 'https://vega.github.io/schema/vega-lite/v5.json', 'data': {'url': 'data/movies.json'}, 'title': 'Relation between Rotten Tomatoes and IMDB rating', 'layer': [{'mark': {'type': 'point', 'filled': True}, 'encoding': {'x': {'field': 'Rotten Tomatoes Rating', 'type': 'quantitative'}, 'y': {'field': 'IMDB Rating', 'type': 'quantitative'}}}, {'mark': {'type': 'text', 'angle': -45}, 'transform': [{'filter': "datum['Rotten Tomatoes Rating'] == 50 && datum['IMDB Rating'] == 6"}], 'encoding': {'text': {'value': '→'}, 'color': {'value': 'red'}, 'x': {'type': 'quantitative', 'field': 'Rotten Tomatoes Rating'}, 'y': {'type': 'quantitative', 'field': 'IMDB Rating'}, 'fontSize': {'value': 100}}}]}


In [114]:
vl_json=json.loads(vl_spec)
vl_json["layer"] = [{ 'mark': vl_json['mark'], 'encoding': vl_json['encoding'] }]
vl_json=ScatterText(vl_json, "Rotten Tomatoes Rating", "IMDB Rating","quantitative","quantitative",50,6,"increase",-45)
del vl_json['mark'], vl_json['encoding']
print(vl_json)
png_data = vlc.vegalite_to_png(vl_spec=vl_json, scale=2)
with open("Trend-Scatter-Text.png", "wb") as f:
	f.write(png_data)

{'$schema': 'https://vega.github.io/schema/vega-lite/v5.json', 'data': {'url': 'data/movies.json'}, 'title': 'Relation between Rotten Tomatoes and IMDB rating', 'layer': [{'mark': {'type': 'point', 'filled': True}, 'encoding': {'x': {'field': 'Rotten Tomatoes Rating', 'type': 'quantitative'}, 'y': {'field': 'IMDB Rating', 'type': 'quantitative'}}}, {'mark': {'type': 'text', 'filled': True, 'fontsize': 100, 'angle': -45, 'dy': -10}, 'transform': [{'filter': "datum['Rotten Tomatoes Rating'] == 50 && datum['IMDB Rating'] == 6"}], 'encoding': {'text': {'value': 'increase'}, 'color': {'value': 'black'}, 'x': {'type': 'quantitative', 'field': 'Rotten Tomatoes Rating'}, 'y': {'type': 'quantitative', 'field': 'IMDB Rating'}, 'fontSize': {'value': 100}}}]}


In [115]:
vl_spec=r''' 
{
  "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
  "description": "A scatterplot showing body mass and flipper lengths of penguins.",
  "data": {
    "url": "data/penguins.json"
  },
  "mark": "point",
  "encoding": {
    "x": {
      "field": "Beak Length (mm)",
      "type": "quantitative",
      "scale": {"zero": false}
    },
    "y": {
      "field": "Body Mass (g)",
      "type": "quantitative",
      "scale": {"zero": false}
    }
  },
  "title":"A scatterplot showing body mass and beak lengths of penguins."
}
'''

In [116]:
png_data = vlc.vegalite_to_png(vl_spec=vl_spec, scale=2)
with open("Sample-Scatter-Compare.png", "wb") as f:
	f.write(png_data)

In [117]:
vl_json=json.loads(vl_spec)
vl_json["layer"] = [{ 'mark': vl_json['mark'], 'encoding': vl_json['encoding'] }]
vl_json=ScatterMark(vl_json, "Beak Length (mm)", "Body Mass (g)", "quantitative", "quantitative", 49.1, 5500)
vl_json=ScatterText(vl_json, "Beak Length (mm)", "Body Mass (g)", "quantitative", "quantitative", 49.1, 5500,'A',dy=5)
vl_json=ScatterMark(vl_json, "Beak Length (mm)", "Body Mass (g)", "quantitative", "quantitative", 40.6, 4000)
vl_json=ScatterText(vl_json, "Beak Length (mm)", "Body Mass (g)", "quantitative", "quantitative", 40.6, 4000,'B',dy=5)
del vl_json['mark'], vl_json['encoding']
print(vl_json)
png_data = vlc.vegalite_to_png(vl_spec=vl_json, scale=2)
with open("Compare-Scatter-Mark.png", "wb") as f:
	f.write(png_data)

{'$schema': 'https://vega.github.io/schema/vega-lite/v5.json', 'description': 'A scatterplot showing body mass and flipper lengths of penguins.', 'data': {'url': 'data/penguins.json'}, 'title': 'A scatterplot showing body mass and beak lengths of penguins.', 'layer': [{'mark': 'point', 'encoding': {'x': {'field': 'Beak Length (mm)', 'type': 'quantitative', 'scale': {'zero': False}}, 'y': {'field': 'Body Mass (g)', 'type': 'quantitative', 'scale': {'zero': False}}}}, {'mark': {'type': 'point', 'filled': True}, 'transform': [{'filter': "datum['Beak Length (mm)'] == 49.1 && datum['Body Mass (g)'] == 5500"}], 'encoding': {'color': {'value': 'red'}, 'size': {'value': 100}, 'x': {'type': 'quantitative', 'field': 'Beak Length (mm)', 'axis': {}, 'bin': False}, 'y': {'type': 'quantitative', 'field': 'Body Mass (g)'}}}, {'mark': {'type': 'text', 'filled': True, 'fontsize': 14, 'angle': 0, 'dy': 5}, 'transform': [{'filter': "datum['Beak Length (mm)'] == 49.1 && datum['Body Mass (g)'] == 5500"}], 

In [118]:
vl_json=json.loads(vl_spec)
vl_json["layer"] = [{ 'mark': vl_json['mark'], 'encoding': vl_json['encoding'] }]
vl_json=ScatterOpacity(vl_json, "Beak Length (mm)", "Body Mass (g)", "quantitative", "quantitative", 49.1,'equal', 5500)
vl_json=ScatterText(vl_json, "Beak Length (mm)", "Body Mass (g)", "quantitative", "quantitative", 49.1, 5500,'A',dy=5)
vl_json=ScatterOpacity(vl_json, "Beak Length (mm)", "Body Mass (g)", "quantitative", "quantitative", 40.6,'equal', 4000)
vl_json=ScatterText(vl_json, "Beak Length (mm)", "Body Mass (g)", "quantitative", "quantitative", 40.6, 4000,'B',dy=5)
del vl_json['mark'], vl_json['encoding']
print(vl_json)
png_data = vlc.vegalite_to_png(vl_spec=vl_json, scale=2)
with open("Compare-Scatter-Opacity.png", "wb") as f:
	f.write(png_data)

{'$schema': 'https://vega.github.io/schema/vega-lite/v5.json', 'description': 'A scatterplot showing body mass and flipper lengths of penguins.', 'data': {'url': 'data/penguins.json'}, 'title': 'A scatterplot showing body mass and beak lengths of penguins.', 'layer': [{'mark': 'point', 'encoding': {'x': {'field': 'Beak Length (mm)', 'type': 'quantitative', 'scale': {'zero': False}}, 'y': {'field': 'Body Mass (g)', 'type': 'quantitative', 'scale': {'zero': False}}, 'opacity': {'value': 0.1}}}, {'mark': 'point', 'encoding': {'x': {'field': 'Beak Length (mm)', 'type': 'quantitative', 'scale': {'zero': False}}, 'y': {'field': 'Body Mass (g)', 'type': 'quantitative', 'scale': {'zero': False}}, 'opacity': {'value': 1}}, 'transform': [{'filter': "datum['Beak Length (mm)'] == 49.1 && datum['Body Mass (g)'] == 5500"}]}, {'mark': {'type': 'text', 'filled': True, 'fontsize': 14, 'angle': 0, 'dy': 5}, 'transform': [{'filter': "datum['Beak Length (mm)'] == 49.1 && datum['Body Mass (g)'] == 5500"}],

In [119]:
vl_json=json.loads(vl_spec)
vl_json["layer"] = [{ 'mark': vl_json['mark'], 'encoding': vl_json['encoding'] }]
vl_json=ScatterColor(vl_json, "Beak Length (mm)", "Body Mass (g)", "quantitative", "quantitative", 49.1,'equal', 5500)
vl_json=ScatterText(vl_json, "Beak Length (mm)", "Body Mass (g)", "quantitative", "quantitative", 49.1, 5500,'A',dy=5)
vl_json=ScatterColor(vl_json, "Beak Length (mm)", "Body Mass (g)", "quantitative", "quantitative", 40.6,'equal', 4000)
vl_json=ScatterText(vl_json, "Beak Length (mm)", "Body Mass (g)", "quantitative", "quantitative", 40.6, 4000,'B',dy=5)
del vl_json['mark'], vl_json['encoding']
print(vl_json)
png_data = vlc.vegalite_to_png(vl_spec=vl_json, scale=2)
with open("Compare-Scatter-Color.png", "wb") as f:
	f.write(png_data)

{'$schema': 'https://vega.github.io/schema/vega-lite/v5.json', 'description': 'A scatterplot showing body mass and flipper lengths of penguins.', 'data': {'url': 'data/penguins.json'}, 'title': 'A scatterplot showing body mass and beak lengths of penguins.', 'layer': [{'mark': 'point', 'encoding': {'x': {'field': 'Beak Length (mm)', 'type': 'quantitative', 'scale': {'zero': False}}, 'y': {'field': 'Body Mass (g)', 'type': 'quantitative', 'scale': {'zero': False}}}}, {'mark': 'point', 'encoding': {'x': {'field': 'Beak Length (mm)', 'type': 'quantitative', 'scale': {'zero': False}}, 'y': {'field': 'Body Mass (g)', 'type': 'quantitative', 'scale': {'zero': False}}, 'color': {'value': 'red'}}, 'transform': [{'filter': "datum['Beak Length (mm)'] == 49.1 && datum['Body Mass (g)'] == 5500"}]}, {'mark': {'type': 'text', 'filled': True, 'fontsize': 14, 'angle': 0, 'dy': 5}, 'transform': [{'filter': "datum['Beak Length (mm)'] == 49.1 && datum['Body Mass (g)'] == 5500"}], 'encoding': {'text': {'v

In [120]:
vl_json=json.loads(vl_spec)
vl_json["layer"] = [{ 'mark': vl_json['mark'], 'encoding': vl_json['encoding'] }]
vl_json=ScatterText(vl_json, "Beak Length (mm)", "Body Mass (g)", "quantitative", "quantitative", 49.1, 5500,'50,5500')
vl_json=ScatterText(vl_json, "Beak Length (mm)", "Body Mass (g)", "quantitative", "quantitative", 49.1, 5500,'A',dy=5)
vl_json=ScatterText(vl_json, "Beak Length (mm)", "Body Mass (g)", "quantitative", "quantitative", 40.6, 4000,'40,4000')
vl_json=ScatterText(vl_json, "Beak Length (mm)", "Body Mass (g)", "quantitative", "quantitative", 40.6, 4000,'B',dy=5)
del vl_json['mark'], vl_json['encoding']
print(vl_json)
png_data = vlc.vegalite_to_png(vl_spec=vl_json, scale=2)
with open("Compare-Scatter-Text.png", "wb") as f:
	f.write(png_data)

{'$schema': 'https://vega.github.io/schema/vega-lite/v5.json', 'description': 'A scatterplot showing body mass and flipper lengths of penguins.', 'data': {'url': 'data/penguins.json'}, 'title': 'A scatterplot showing body mass and beak lengths of penguins.', 'layer': [{'mark': 'point', 'encoding': {'x': {'field': 'Beak Length (mm)', 'type': 'quantitative', 'scale': {'zero': False}}, 'y': {'field': 'Body Mass (g)', 'type': 'quantitative', 'scale': {'zero': False}}}}, {'mark': {'type': 'text', 'filled': True, 'fontsize': 14, 'angle': 0, 'dy': -10}, 'transform': [{'filter': "datum['Beak Length (mm)'] == 49.1 && datum['Body Mass (g)'] == 5500"}], 'encoding': {'text': {'value': '50,5500'}, 'color': {'value': 'black'}, 'x': {'type': 'quantitative', 'field': 'Beak Length (mm)'}, 'y': {'type': 'quantitative', 'field': 'Body Mass (g)'}, 'fontSize': {'value': 14}}}, {'mark': {'type': 'text', 'filled': True, 'fontsize': 14, 'angle': 0, 'dy': 5}, 'transform': [{'filter': "datum['Beak Length (mm)']